In [2]:
%matplotlib widget
%reload_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
import tdt
from pathlib import Path
import matplotlib.pyplot as plt
import seaborn as sns
import yaml
import xarray as xr

import kdephys.hypno as kh
import kdephys.pd as kp
import kdephys.xr as kx
import kdephys.utils as ku
import kdephys.ssfm as ss
import ipywidgets as wd
import acr.subjects as asub

import acr

import plotly.express as px
import plotly.io as pio
pio.templates.default = "plotly_dark"

bands = ku.spectral.bands

In [19]:
acr9_info = {}
acr9_info['subject'] = 'ACR_9'
acr9_info['complete_key_list'] = ['control1', 'laser1']

In [10]:
def save_dataframes(df_dict, si, type='-bp'):
    sub = si['subject']
    save_path = '/Volumes/opto_loc/Data/ACR_PROJECT_MATERIALS/'+sub+'/'+'analysis-data/'
    for key in list(df_dict.keys()):
        df_dict[key].to_pickle(save_path+key+type+'.pkl')
        print(f'{key} saved')
    return None

In [11]:
def add_time_class(df, times):
    if 'control1' in df.condition[0]:
        stim_on = times['control1']['stim_on_dt']
        stim_off = times['control1']['stim_off_dt']
    elif 'laser1' in df.condition[0]:
        stim_on = times['laser1']['stim_on_dt']
        stim_off = times['laser1']['stim_off_dt']
    df['time_class'] = 'NA'
    df.loc[df.datetime.between(df.datetime.min(), stim_on), 'time_class'] = 'Baseline'
    stim_mid = stim_on + pd.Timedelta('2H')
    df.loc[df.datetime.between(stim_on, stim_mid), 'time_class'] = 'Photostim, 0-2Hr'
    df.loc[df.datetime.between(stim_mid, stim_off), 'time_class'] = 'Photostim, 2-4Hr'
    c1 = stim_off+pd.Timedelta('2H')
    c2 = stim_off+pd.Timedelta('4H')
    c3 = stim_off+pd.Timedelta('6H')
    df.loc[df.datetime.between(stim_off, c1), 'time_class'] = 'Post Stim, 0-2Hr'
    df.loc[df.datetime.between(c1, c2), 'time_class'] = 'Post Stim, 2-4Hr'
    df.loc[df.datetime.between(c2, c3), 'time_class'] = 'Post Stim, 4-6Hr'
    return df

# ACR_9

In [142]:
import acr.acr_info as ai
t = ai.a9_times
control_laser_on = t['control1']['stim_on_dt']
control_laser_off = t['control1']['stim_off_dt']
laser_laser_on = t['laser1']['stim_on_dt']
laser_laser_off = t['laser1']['stim_off_dt']

In [ ]:
a9_times_c1 = {}
a9_times_c1['bl_sleep_start'] = 5753
a9_times_c1['stim_on'] = 6574
a9_times_c1['stim_off'] = 20974
a9_times_c1['stim_on_dt'] = pd.Timestamp('2022-06-14 10:40:27.383717400')
a9_times_c1['stim_off_dt'] = pd.Timestamp('2022-06-14 14:40:27.385437720')
a9_times_l1 = {}
a9_times_l1['bl_sleep_start'] = 9107
a9_times_l1['stim_on'] = 9921
a9_times_l1['stim_off'] = 24321
a9_times_l1['stim_on_dt'] = pd.Timestamp('2022-06-17 11:18:33.631271960')
a9_times_l1['stim_off_dt'] = pd.Timestamp('2022-06-17 15:18:33.632992280')
a9_times = {}
a9_times['control1'] = a9_times_c1
a9_times['laser1'] = a9_times_l1

In [ ]:
key_list = acr9_info['complete_key_list']
pth = acu.get_paths(acr9_info['subject'], key_list)

In [16]:
a9 = {}
a9['control1-eeg'] = kx.io.get_data(pth['control1'], 'EEGr', t1=5723, t2=42575, channel=[1,2])
a9['laser1-eeg'] = kx.io.get_data(pth['laser1'], 'EEGr', t1=9077, t2=45922, channel=[1,2])
a9['control1-lfp'] = kx.io.get_data(pth['control1'], 'LFP_', t1=5723, t2=42575, channel=[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16])
a9['laser1-lfp'] = kx.io.get_data(pth['laser1'], 'LFP_', t1=9077, t2=45922, channel=[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16])

read from t=5723s to t=44120.52s
Using 1017.2526 Hz as SEV sampling rate for EEGr
read from t=9077s to t=47213.6s
Using 1017.2526 Hz as SEV sampling rate for EEGr
read from t=5723s to t=44120.52s
Using 610.3516 Hz as SEV sampling rate for LFP_
read from t=9077s to t=47213.6s
Using 610.3516 Hz as SEV sampling rate for LFP_


In [19]:
control_start = a9['control1-eeg'].datetime.values.min()
laser_start = a9['laser1-eeg'].datetime.values.min()
a9_hyp = {}
a9_hyp['control1'] = load_hypno('ACR_9', 'control1', control_start)
a9_hyp['laser1'] = load_hypno('ACR_9', 'laser1', laser_start)

In [20]:
a9_spg = kx.spectral.get_spg_from_dataset(a9)
a9_spg = add_hypnograms_to_dataset(a9_spg, a9_hyp)


for key in a9_spg.keys():
    a9_spg[key] = xarray_to_pandas(a9_spg[key], name='spg')
    a9_spg[key] = redo_timdelta(a9_spg[key])
a9_bp = {}
for key in list(a9_spg.keys()):
    a9_bp[key] = kx.spectral.get_bp_set(a9_spg[key], bands)
a9_bp = add_hypnograms_to_dataset(a9_bp, a9_hyp)

for key in list(a9_bp.keys()):
    a9_bp[key] = xarray_to_pandas(a9_bp[key])
    a9_bp[key] = redo_timdelta(a9_bp[key])
    a9_bp[key]['condition'] = key
    a9_bp[key] = a9_bp[key].reset_index()

In [44]:
save_path = '/Volumes/opto_loc/Data/ACR_PROJECT_MATERIALS/ACR_9/analysis-data/'
save_dataframes(a9_bp, save_path, type='-bp')

control1-eeg saved
laser1-eeg saved
control1-lfp saved
laser1-lfp saved


In [ ]:
save_hypnograms(a9_hyp, save_path)

In [ ]:

save_dataframes(a9_spg, save_path, type='-spg')

# ACR_10

In [38]:
a10_info = {}
a10_info['subject'] = 'ACR_10'
a10_info['complete_key_list'] = ['laser1', 'laser1-bl']
a10_info['paths'] = acu.get_paths(a10_info['subject'], a10_info['complete_key_list'])
a10_info['start_times'] = {}
a10_info['start_times']['laser1'] = 4158
a10_info['start_times']['laser1-bl'] = 0

In [16]:
a10 = {}
a10['laser1'] = kx.io.get_data(a10_info['paths']['laser1'], 'EEGr', t1=4158, t2=4158+14400, channel=[1,2])
a10['laser1-bl'] = kx.io.get_data(a10_info['paths']['laser1-bl'], 'EEGr', t1=0, t2=14400, channel=[1,2])

read from t=4158s to t=19129.73s
Using 1017.2526 Hz as SEV sampling rate for EEGr
read from t=0s to t=21096.09s
Using 1017.2526 Hz as SEV sampling rate for EEGr


In [29]:
a10_hyp = load_hypno(a10_info, a10)

In [43]:
a10_spg, a10_bp = get_spectral(a10_info, a10, a10_hyp)

In [48]:
save_dataframes(a10_bp, a10_info, type='-bp')
save_dataframes(a10_hyp, a10_info, type='-hyp')

laser1 saved
laser1-bl saved


# ACR_11

In [33]:
a11_info = {}
a11_info['subject'] = 'ACR_11'
a11_info['complete_key_list'] = ['laser1', 'laser1-bl']
a11_info['paths'] = acu.get_paths(a11_info['subject'], a11_info['complete_key_list'])
a11_info['start_times'] = {}
a11_info['start_times']['laser1'] = 5974
a11_info['start_times']['laser1-bl'] = 0

In [15]:
a11 = {}
a11['laser1'] = kx.io.get_data(a11_info['paths']['laser1'], 'EEGr', t1=5974, t2=5974+14400, channel=[1,2])
a11['laser1-bl'] = kx.io.get_data(a11_info['paths']['laser1-bl'], 'EEGr', t1=0, t2=14400, channel=[1,2])

read from t=5974s to t=21096.01s
Using 1017.2526 Hz as SEV sampling rate for EEGr
read from t=0s to t=21096.01s
Using 1017.2526 Hz as SEV sampling rate for EEGr


In [30]:
a11_hyp = load_hypno(a11_info, a11)

In [45]:
a11_spg, a11_bp = get_spectral(a11_info, a11, a11_hyp)

In [51]:
save_dataframes(a11_bp, a11_info, type='-bp')
save_dataframes(a11_hyp, a11_info, type='-hyp')

laser1 saved
laser1-bl saved
laser1 saved
laser1-bl saved


# ACR_12

In [3]:
a12_info = asub.a12_info

In [ ]:
a12, spg, bp, hyp = acr.io.acr_load_master(a12_info, stores=['EEGr', 'NNXr'])

# Misc. Functions

In [ ]:
def get_rel_bp_set(df, laser_on):
    bl_period = slice(df.datetime.min(), laser_on)
    bl_df = df.ts(bl_period)
    df = df.set_index(['datetime', 'channel'])
    bl_df.set_index(['datetime', 'channel'], inplace=True)
    means = bl_df.groupby(['channel']).mean()
    for col in list(means.columns):
        df[col] = df[col]/means[col]
    df = df.reset_index()
    return df

In [ ]:
def means_meds(bp_set, times, condition, dtype, channel=1, periods=15, band='delta', state=['NREM']):
    bp = bp_set[condition+dtype].ch(channel)
    if state:
        bp = bp.filt_state(states=state)
    time_interval = pd.date_range(times[condition]['stim_on_dt'], bp.datetime.max(), periods=periods)
    mn = []
    md = []
    for i in range(0, len(time_interval)-1):
        mn.append(bp.ts(slice(time_interval[i], time_interval[i+1]))[band].mean())
        md.append(bp.ts(slice(time_interval[i], time_interval[i+1]))[band].median())
    vals = pd.DataFrame(mn, index=time_interval[1:periods], columns=[band+'_mean'])
    vals[band+'_median'] = md
    vals['condition'] = condition
    vals['count'] = np.arange(0, len(vals))
    vals['timedelta'] = (vals.index.values - np.datetime64(times[condition]['stim_on_dt'])).astype('timedelta64[m]')
    return vals